# MMS dayside magnetopause

This notebook performs analysis on MMS (Magnetospheric Multiscale) spacecraft data, specifically for MMS2. It includes:

1. Walén Test Analysis
   - Calculates the Walén relation comparing plasma flow velocity to the Alfvén velocity
   - Uses ion bulk velocity, magnetic field, and density measurements

2. Mirror Mode Analysis  
   - Calculates the mirror mode criterion (Cm)
   - Uses magnetic field, temperature anisotropy and plasma parameters

3. Data Visualization
   - Energy spectrograms
   - Velocity components
   - Mirror mode criterion
   - Walén test results
   - Magnetic field components
   - Temperature (parallel/perpendicular)
   - Density measurements

The notebook utilizes both survey and burst mode data from the FPI (Fast Plasma Investigation) and FGM (Fluxgate Magnetometer) instruments.

Time period analyzed: November 18, 2015 02:14:30 - 03:34:00 UT


In [ ]:
from datetime import datetime
import speasy as spz
import numpy as np
import scipy
import scipy.constants as cst

from SciQLop.user_api.plot import create_plot_panel, TimeRange, ScaleType
from SciQLop.user_api.virtual_products import create_virtual_product, VirtualProductType

def mms2_walen(start: datetime, stop: datetime) -> (np.ndarray, np.ndarray):

    v = spz.get_data(
        spz.inventories.data_tree.cda.MMS.MMS2.DIS.MMS2_FPI_FAST_L2_DIS_MOMS.mms2_dis_bulkv_gse_fast,
        start,
        stop,
    )
    b = spz.get_data(
        spz.inventories.tree.cda.MMS.MMS2.FGM.MMS2_FGM_SRVY_L2.mms2_fgm_b_gsm_srvy_l2,
        start,
        stop,
    )
    N = spz.get_data(
        spz.inventories.data_tree.cda.MMS.MMS2.DIS.MMS2_FPI_FAST_L2_DIS_MOMS.mms2_dis_numberdensity_fast,
        start,
        stop,
    )
    vx = v["Vx_GSE"].values[:, 0]
    vy = v["Vy_GSE"].values[:, 0]
    vz = v["Vz_GSE"].values[:, 0]

    t = v["Vx_GSE"].time.astype(np.timedelta64) / np.timedelta64(1, "s")
    tb = b["Bt"].time.astype(np.timedelta64) / np.timedelta64(1, "s")
    Nfunc = scipy.interpolate.interp1d(
        t, N.values[:, 0], bounds_error=False, fill_value="extrapolate"
    )
    NonB = Nfunc(tb)

    Vax = b["Bx GSM"].values[:, 0] * 1e-9 / np.sqrt(cst.mu_0 * cst.m_p * NonB * 1e6)
    Vay = b["By GSM"].values[:, 0] * 1e-9 / np.sqrt(cst.mu_0 * cst.m_p * NonB * 1e6)
    Vaz = b["Bz GSM"].values[:, 0] * 1e-9 / np.sqrt(cst.mu_0 * cst.m_p * NonB * 1e6)

    imid = int(Vax.size / 2.0)
    vxl = vx[0] * 1e3 + Vax[:imid] - Vax[0]
    vyl = vy[0] * 1e3 + Vay[:imid] - Vay[0]
    vzl = vz[0] * 1e3 + Vaz[:imid] - Vaz[0]
    vxr = vx[-1] * 1e3 + Vax[imid:] - Vax[-1]
    vyr = vy[-1] * 1e3 + Vay[imid:] - Vay[-1]
    vzr = vz[-1] * 1e3 + Vaz[imid:] - Vaz[-1]


    vw = np.zeros((Vax.size, 3))
    vw[:imid, 0] = vxl / 1e3
    vw[imid:, 0] = vxr / 1e3

    vw[:imid, 1] = vyl / 1e3
    vw[imid:, 1] = vyr / 1e3

    vw[:imid, 2] = vzl / 1e3
    vw[imid:, 2] = vzr / 1e3

    return tb, vw


def mms2_mirror_cm(start: datetime, stop: datetime) -> (np.ndarray, np.ndarray):
    b = spz.get_data(
        spz.inventories.tree.cda.MMS.MMS2.FGM.MMS2_FGM_SRVY_L2.mms2_fgm_b_gsm_srvy_l2,
        start,
        stop,
    )
    Tperp = spz.get_data(
        spz.inventories.data_tree.cda.MMS.MMS2.DIS.MMS2_FPI_FAST_L2_DIS_MOMS.mms2_dis_tempperp_fast,
        start,
        stop,
    )
    Tpara = spz.get_data(
        spz.inventories.data_tree.cda.MMS.MMS2.DIS.MMS2_FPI_FAST_L2_DIS_MOMS.mms2_dis_temppara_fast,
        start,
        stop,
    )
    N = spz.get_data(
        spz.inventories.data_tree.cda.MMS.MMS2.DIS.MMS2_FPI_FAST_L2_DIS_MOMS.mms2_dis_numberdensity_fast,
        start,
        stop,
    )
    anisotropy = Tperp["eT_perp"].values / Tpara["eT_para"].values
    bt = b["Bt"].values
    Pperp = N["N"].values * Tperp["eT_perp"].values
    tb = b["Bt"].time.astype(np.timedelta64) / np.timedelta64(1, "s")
    tp = N.time.astype(np.timedelta64) / np.timedelta64(1, "s")
    pfunc = scipy.interpolate.interp1d(tp, Pperp[:, 0], bounds_error=False)
    Afunc = scipy.interpolate.interp1d(tp, anisotropy[:, 0], bounds_error=False)
    betaperp = pfunc(tb) * 1e6 * cst.e * 2 * cst.mu_0 / (bt[:, 0] * 1e-9) ** 2
    cm = betaperp * (Afunc(tb) - 1)
    cm = scipy.ndimage.gaussian_filter1d(cm, 6)
    return tb, cm


cm_provider = create_virtual_product(
    path="mms_vprods/mms2/mirror_cm",
    callback=mms2_mirror_cm,
    product_type=VirtualProductType.Scalar,
    labels=["C_M_b"],
    cachable=True
)

provider = create_virtual_product(
    path="mms_vprods/mms2/vwalen",
    callback=mms2_walen,
    product_type=VirtualProductType.Vector,
    labels=["x", "y", "z"],
    cachable=True
)


# all plots are stacked
p = create_plot_panel()
p.time_range = TimeRange(
    datetime(2015, 11, 18, 2, 14, 30).timestamp(),
    datetime(2015, 11, 18, 3, 34, 0).timestamp(),
)

plot_names = ["spectro_fast", "V", "cm", "Walen","fgm", "tB", "n"]
plots_idx = {plot_name: i for plot_name, i in zip(plot_names, range(len(plot_names)))}

p.plot(
    "speasy/cda/MMS/MMS2/DIS/MMS2_FPI_FAST_L2_DIS_MOMS/mms2_dis_energyspectr_omni_fast"
)

p.plot("speasy/cda/MMS/MMS2/DIS/MMS2_FPI_FAST_L2_DIS_MOMS/mms2_dis_bulkv_gse_fast")
p.plots[plots_idx["V"]].plot(
    "speasy/cda/MMS/MMS2/DIS/MMS2_FPI_BRST_L2_DIS_MOMS/mms2_dis_bulkv_gse_brst"
)

p.plot("mms_vprods/mms2/mirror_cm")
p.plot("mms_vprods/mms2/vwalen")

p.plot("speasy/cda/MMS/MMS2/FGM/MMS2_FGM_SRVY_L2/mms2_fgm_b_gsm_srvy_l2")

p.plot("speasy/cda/MMS/MMS2/DIS/MMS2_FPI_FAST_L2_DIS_MOMS/mms2_dis_temppara_fast")
p.plots[plots_idx["tB"]].plot(
    "speasy/cda/MMS/MMS2/DIS/MMS2_FPI_FAST_L2_DIS_MOMS/mms2_dis_tempperp_fast"
)
p.plot("speasy/cda/MMS/MMS2/DIS/MMS2_FPI_FAST_L2_DIS_MOMS/mms2_dis_numberdensity_fast")

p.plots[plots_idx["n"]].plot(
    "speasy/cda/MMS/MMS2/DIS/MMS2_FPI_BRST_L2_DIS_MOMS/mms2_dis_numberdensity_brst"
)


p.plots[plots_idx["tB"]].set_y_range(180, 1500)
p.plots[plots_idx["fgm"]].set_y_range(-75, 75)
p.plots[plots_idx["n"]].set_y_range(0.001, 50)
p.plots[plots_idx["cm"]].set_y_range(-4, 6)
p.plots[plots_idx["Walen"]].set_y_range(-300, 300)
p.plots[plots_idx["V"]].set_y_range(-450, 450)
p.plots[plots_idx["tB"]].set_y_scale_type(ScaleType.Logarithmic)
p.plots[plots_idx["n"]].set_y_scale_type(ScaleType.Logarithmic)
p.plots[plots_idx["tB"]].set_y_scale_type(ScaleType.Logarithmic)
p.plots[plots_idx["n"]].set_y_scale_type(ScaleType.Logarithmic)
